# Transfer learning

Ce notebook télécharge le modèle RoBERTa-base et utilise le dataset MultiNERD English pour prédire les noms de personnes sur du texte anglais. <br/>
Il faut:
- L'adapter sur du français (modèle camembert, autre dataset)
- Essayer de freeze des layers, améliorer ses performances sur le jeu "dev" 

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import torch
import transformers

In [ ]:
model_name ="camembert-base"

## MultiNERD data

Ce dataset est un text avec des catégories assez fines (dont nom de personne).<br>
Il est disponible [ici](https://github.com/Babelscape/multinerd)<br>
Prenez le dataset français<br>

Importation de la data 30% test 
Nous avons tester avec 20% et 2% pour êtres plus rapide

In [ ]:
with open("./Data/train_fr.tsv") as f:
    all_rows = list(line.strip().split("\t") for line in f)
    # Select  50% of the rows
    rows = all_rows[:int(len(all_rows) * 0.30 )]

rows[:10]

 fonction make_labelled_sentences est conçue pour transformer une liste de mots étiquetés 

Si le mot actuel est un point (.), cela signifie la fin d'une phrase. La liste actuelle de mots (this_word) est ajoutée à X et la liste d'étiquettes correspondantes (this_labels) est ajoutée à y. Ensuite, this_word et this_labels sont réinitialisés pour la prochaine phrase.
Si le mot n'est pas un point, il est ajouté à la liste des mots actuelle (this_word). L'étiquette est déterminée en vérifiant si l'étiquette du mot se termine par "PER". Si c'est le cas, 1 est ajouté à this_labels (indiquant une entité nommée de personne), sinon 0 est ajouté.

In [ ]:
def make_labelled_sentences(tagged_words):
    # Joining words until we meet a dot
    # Word's label is 1 if 'PER' is in its tag
    X = []
    y = []

    this_word = []
    this_labels = []
    for tagged_word in tagged_words:
        if len(tagged_word) < 3:
            # not a tagged word
            continue
        word = tagged_word[1]
        tag = tagged_word[2]

        if word == '.':
            X.append(this_word)
            y.append(this_labels)

            this_word = []
            this_labels = []
        else:
            this_word.append(word)
            this_labels.append(1 * tag.endswith("PER"))

    return X, y

In [ ]:
sentences, labels = make_labelled_sentences(rows) #donnée brut -> utilisable

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
sentences_training, sentences_test, labels_training, labels_test = train_test_split(
    sentences,
    labels,
    test_size=0.2,
    random_state=42,
) #split données

In [ ]:
sentences_train, sentences_dev, labels_train, labels_dev = train_test_split(
    sentences_training,
    labels_training,
    test_size=0.2,
    random_state=42,
)

In [ ]:
import sentencepiece

# Applying Hugging face V2

In [ ]:
from transformers import CamembertTokenizerFast, CamembertForTokenClassification,RobertaConfig

#definition du modèle camembert
num_labels = 31
config = RobertaConfig.from_pretrained(model_name, num_hidden_layers=8)  # Ajustez le nombre de couches cachées
#model = CamembertForTokenClassification.from_pretrained("camembert-base", num_labels=num_labels)
model = CamembertForTokenClassification.from_pretrained("camembert-base", config=config)
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")




Nous regardons un peu les couches leurs type, les sorties ainsi que les différents paramètres

In [ ]:
# Analyse détaillée des couches du modèle
for idx, (name, layer) in enumerate(model.named_children()):
    print(f"Couche {idx}: {name}")
    print(f"Type de Couche: {type(layer).__name__}")
    print(f"Forme de Sortie: {layer.out_features if hasattr(layer, 'out_features') else 'N/A'}")
    print(f"Paramètres Entraînables: {sum(p.numel() for p in layer.parameters() if p.requires_grad)}")
    print(f"Paramètres Non-Entraînables: {sum(p.numel() for p in layer.parameters() if not p.requires_grad)}\n")


nous regardons les noms des couches mis en commentaire si besoin

In [ ]:
'''for name, param in model.named_parameters():
    print(name)
'''

Geler les paramètres nous avons d'abord essayer en gélens tous les couches sauf la dernière pour voir ce qui ce passe puie en fesant des nombre de couche réduit , afficher les couches gelées

In [ ]:
'''# Geler tous les paramètres
for param in model.roberta.parameters():
    param.requires_grad = False

# Dégelez les derniers layers (par exemple, les deux dernières couches)
for param in model.roberta.encoder.layer[-2:].parameters():
    param.requires_grad = True

# Assurez-vous que le classificateur est également dégelé
for param in model.classifier.parameters():
    param.requires_grad = True'''
# Geler toutes les couches sauf les dernières
nombre_de_couches_a_geler = 10  # Ajustez ce nombre selon vos besoins
for layer in model.roberta.encoder.layer[:-nombre_de_couches_a_geler]:
    for param in layer.parameters():
        param.requires_grad = False

# Vérifier les paramètres après le gel
for name, param in model.named_parameters():
    print(f"{name} est {'gelé' if not param.requires_grad else 'dégelé'}")




cette fonction def tokenize_and_align_labels(sentences, ner_tags) prépare les données pour l'entraînement ou l'évaluation d'un modèle de classification de tokens en NLP, en s'assurant que chaque token reçoit l'étiquette correcte

In [ ]:
'''def tokenize_and_align_labels(sentences, ner_tags):
    tokenized_inputs = tokenizer(
        sentences,
        truncation=True,
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(ner_tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs'''
    
def tokenize_and_align_labels(sentences, ner_tags):
    tokenized_inputs = tokenizer(
        sentences,
        truncation=True,
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(ner_tags):
        print(f"Processing sentence {i}, Length of labels: {len(label)}")  # Débogage

        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            print(f"Word index: {word_idx}")  # Débogage

            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                if word_idx < len(label):
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
            else:
                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs


In [ ]:
#on regarde 5 éléments 
for i in range(5):
    print(sentences_train[i])
    print(labels_train[i])
    print()


In [ ]:
# vérifier la validité des données dans sentences_train
for i, sentence in enumerate(sentences_train):
    if not sentence or isinstance(sentence, str):
        print(f"Problème avec la phrase à l'indice {i}: {sentence}")


In [ ]:
filtered_sentences_train = []
filtered_labels_train = []

for sentence, label in zip(sentences_train, labels_train):
    if sentence:  # Vérifiez si la liste de mots n'est pas vide
        filtered_sentences_train.append(sentence)
        filtered_labels_train.append(label)


utilisation de la fonction pour tokeniser les phrases et aligner les étiquettes avec les tokens générés

In [ ]:
#tokenized_train = tokenize_and_align_labels(sentences_train, labels_train)
tokenized_train = tokenize_and_align_labels(filtered_sentences_train, filtered_labels_train)


In [ ]:
tokenized_test = tokenize_and_align_labels(sentences_test, labels_test)

In [ ]:
from datasets import Dataset
# convertit un dictionnaire de données en un dataset
dataset_train = Dataset.from_dict(tokenized_train)
dataset_test = Dataset.from_dict(tokenized_test)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

compute_metrics est conçue pour calculer les métriques d'évaluation pour une tâche de classification de tokens, telles que la précision, le rappel, le score F1 et l'exactitude. 

La fonction retourne un dictionnaire contenant les métriques calculées.

In [ ]:
'''import numpy as np
import evaluate

seqeval = evaluate.load("seqeval")

labels = [0, 1]
label_list = ["0", "1"]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
'''


import numpy as np
import evaluate

# Chargement de seqeval pour le calcul des métriques
seqeval = evaluate.load("seqeval")

# Liste des étiquettes pour une tâche binaire
label_list = ["0", "1"]

def compute_metrics(p):
    predictions, labels = p
    #print(predictions,labels)
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    
    #print("pred",true_predictions)
    #print("lab",true_labels)

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }





Verification CUDA

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)
print(torch.version.cuda)


In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
'''import torch
print(torch.cuda.is_available())
print(torch.__version__)
print(torch.version.cuda)'''


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

#charger et configurer un modèle de classification de tokens.

model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=2
)
model = model.to("cuda")

In [ ]:
for name, _ in model.base_model.named_parameters():
  print(name)

geler les couches du model

Test nb couche=6,8,10,11

In [ ]:
# for name, param in model.base_model.named_parameters():
#   param.requires_grad = False
# 
# for name, param in model.base_model.named_parameters():
#     if (
#         any(layer_name in name for layer_name in ["layer.5"])
#         and any(layer_type in name for layer_type in ["weight", "bias"])
#         and "attention" not in name
#     ):
#         param.requires_grad = True

# Geler les premières N couches du modèle
N = 8  # exemple : geler les 6 premières couches
for i, (name, param) in enumerate(model.base_model.named_parameters()):
    layer_num = name.split(".")[2] if "layer" in name else None
    if layer_num and int(layer_num) < N:
        param.requires_grad = False

Construction des plots pour afficher les metrics 

In [ ]:
#def plots metrics

import matplotlib.pyplot as plt

def plot_metrics(history):
    plt.figure(figsize=(15, 5))

    # Précision
    plt.subplot(1, 4, 1)
    plt.plot(history['precision'], label='Précision')
    plt.title('Précision par époque')
    plt.legend()

    # Rappel
    plt.subplot(1, 4, 2)
    plt.plot(history['recall'], label='Rappel')
    plt.title('Rappel par époque')
    plt.legend()

    # Score F1
    plt.subplot(1, 4, 3)
    plt.plot(history['f1'], label='Score F1')
    plt.title('Score F1 par époque')
    plt.legend()

    # Exactitude
    plt.subplot(1, 4, 4)
    plt.plot(history['accuracy'], label='Exactitude')
    plt.title('Exactitude par époque')
    plt.legend()

    plt.show()

# Initialisation d'un dictionnaire pour stocker les métriques
metrics_history = {'precision': [], 'recall': [], 'f1': [], 'accuracy': []}


configure les arguments d'entraînement pour un modèle de classification de tokens et initialise un entraîneur (Trainer) pour gérer l'entraînement et l'évaluation du modèle

test avec plusieurs epoch 2,3,10,15

In [ ]:
training_args = TrainingArguments(
    output_dir="MonModels/",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



Calcule des metrics

In [ ]:
trainer.train()

# Enregistrement des métriques après chaque époque
for log in trainer.state.log_history:
    if 'eval_loss' in log.keys():
        metrics_history['precision'].append(log['eval_precision'])
        metrics_history['recall'].append(log['eval_recall'])
        metrics_history['f1'].append(log['eval_f1'])
        metrics_history['accuracy'].append(log['eval_accuracy'])

affichage des résultat graphique des metrics

In [ ]:
# Affichage des graphiques
plot_metrics(metrics_history)